In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
import numpy as np
import pandas as pd
import glob
from datetime import datetime as datetime
from datetime import timedelta
pd.options.mode.chained_assignment = None

In [4]:
lakes_folder = "./lakes/"

START = datetime.strptime("2000-01-04", "%Y-%m-%d") # one year before the first VHI value (2001/01/05)
END = datetime.strptime("2019-12-31", "%Y-%m-%d")
weeks = pd.date_range(start=START, end=END, freq = '8D') # 8 days weeks in line with VHI target data
dates_8days = pd.date_range(start=START, end=END, freq = '8D')

num_months = len(pd.date_range(START, END,freq='MS'))
num_days = (END - START).days
num_weeks = len(weeks)
print(num_months)
print(num_weeks)
print(num_days)

239
913
7301


### Extract data from matlab

In [5]:
mat_files_lakes = glob.glob(lakes_folder + '*.mat')
mat_files_lakes.sort()
mat_files_lakes

['./lakes/a_Mio.mat',
 './lakes/a_PTre.mat',
 './lakes/h_PTre.mat',
 './lakes/h_SC.mat',
 './lakes/r_Mio.mat',
 './lakes/r_PTre.mat']

In [6]:
def matlab_datenum_to_datetime(matlab_datenum):
    python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
    return python_datetime

In [7]:
def filter_date(df):
  df = df.loc[(df['date'] >= START) & (df['date'] <= END)]
  df.reset_index(inplace = True, drop = True)
  return df

In [8]:
import scipy.io

for mat_file in mat_files_lakes:
  mat = scipy.io.loadmat(mat_file)
  mat = mat['Kali_tmp']
  mdtype = mat.dtype
  ndata = {n: mat[n][0][0] for n in mdtype.names}
  
  if "a_Mio" in mat_file:
    dates = ndata['signal'][0][0][2]
    dates = [i[0] for i in dates]
    values = ndata['signal'][0][0][3].ravel()
    dt_dates = np.array([matlab_datenum_to_datetime(date) for date in dates])
    inflow_Maggiore = pd.DataFrame({'date': dt_dates, 'inflow': values})
  elif "a_PTre" in mat_file:
    dates = ndata['signal'][0][0][2]
    dates = [i[0] for i in dates]
    values = ndata['signal'][0][0][3].ravel()
    dt_dates = np.array([matlab_datenum_to_datetime(date) for date in dates])
    inflow_Lugano_PonteTresa = pd.DataFrame({'date': dt_dates, 'inflow': values})
  elif "h_PTre" in mat_file:
    dates = ndata['signal'][0][0][1]
    dates = [i[0] for i in dates]
    values = ndata['signal'][0][0][0].ravel()
    dt_dates = np.array([matlab_datenum_to_datetime(date) for date in dates])
    level_Lugano_PonteTresa = pd.DataFrame({'date': dt_dates, 'level': values})
  elif "h_SC" in mat_file:
    dates = ndata['signal'][0][0][1]
    dates = [i[0] for i in dates]
    values = ndata['signal'][0][0][0].ravel()
    dt_dates = np.array([matlab_datenum_to_datetime(date) for date in dates])
    level_Maggiore = pd.DataFrame({'date': dt_dates, 'level': values})
  elif "r_Mio" in mat_file:
    dates = ndata['signal'][0][0][1]
    dates = [i[0] for i in dates]
    values = ndata['signal'][0][0][0].ravel()
    dt_dates = np.array([matlab_datenum_to_datetime(date) for date in dates])
    release_Maggiore = pd.DataFrame({'date': dt_dates, 'release': values})
  elif "r_PTre" in mat_file:
    dates = ndata['signal'][0][0][1]
    dates = [i[0] for i in dates]
    values = ndata['signal'][0][0][0].ravel()
    dt_dates = np.array([matlab_datenum_to_datetime(date) for date in dates])
    release_Lugano_PonteTresa = pd.DataFrame({'date': dt_dates, 'release': values})
  
inflow_Maggiore = filter_date(inflow_Maggiore)
inflow_Lugano_PonteTresa = filter_date(inflow_Lugano_PonteTresa)
level_Lugano_PonteTresa = filter_date(level_Lugano_PonteTresa)
level_Maggiore = filter_date(level_Maggiore)
release_Maggiore = filter_date(release_Maggiore)
release_Lugano_PonteTresa = filter_date(release_Lugano_PonteTresa)

In [9]:
df_Maggiore = pd.concat([level_Maggiore.set_index('date'), release_Maggiore.set_index('date'), 
                     inflow_Maggiore.set_index('date')], axis=1).reset_index()
df_Maggiore = df_Maggiore.add_suffix("_Maggiore")

df_Lugano = pd.concat([level_Lugano_PonteTresa.set_index('date'), release_Lugano_PonteTresa.set_index('date'), 
                     inflow_Lugano_PonteTresa.set_index('date')], axis=1).reset_index()

df_Lugano = df_Lugano.add_suffix("_Lugano")

df_Maggiore.rename(columns={'date_Maggiore': 'date'}, inplace= True)
df_Lugano.rename(columns={'date_Lugano': 'date'}, inplace= True)

In [10]:
df_Lugano

,date,level_Lugano,release_Lugano,inflow_Lugano
0,2000-01-04,270.4900,9.541417,7.277528
1,2000-01-05,270.4860,9.479354,7.781438
2,2000-01-06,270.4830,9.391882,8.825910
3,2000-01-07,270.4820,9.244486,5.848653
4,2000-01-08,270.4760,9.126597,8.560625
...,...,...,...,...
7297,2019-12-27,270.8885,55.850792,41.984472
7298,2019-12-28,270.8640,51.665771,35.252576
7299,2019-12-29,270.8350,48.688125,31.142986
7300,2019-12-30,270.8040,39.226271,31.302660


### Extract data from excel file (Iseo)

In [11]:
def fixExceldf(df, varname):
  df = pd.melt(df, id_vars = ['Giorno'], var_name = 'year', value_name = varname)
  # remove NaN values that are present only in wrong dates (02-29)
  df = df.dropna(subset=[varname])
  df.reset_index(drop=True, inplace=True)
  df['year'] = df['year'].astype(str)
  df['date'] = df[['year', 'Giorno']].apply(lambda x: '-'.join(x), axis=1)
  df['date'] = pd.to_datetime(df['date'])
  df = filter_date(df)
  df = df[['date', varname]]
  return df

In [12]:
excel_file = pd.ExcelFile(lakes_folder + 'LagoIseo_Dati_Storici.xlsx')

level_df = pd.read_excel(excel_file, 'Altezze cm')
release_df = pd.read_excel(excel_file, 'Deflusso')
inflow_df = pd.read_excel(excel_file, 'Afflusso')

level_df_fixed = fixExceldf(level_df, 'level')
release_df_fixed = fixExceldf(release_df, 'release')
inflow_df_fixed = fixExceldf(inflow_df, 'inflow')

In [13]:
df_Iseo = pd.concat([level_df_fixed.set_index('date'), release_df_fixed.set_index('date'), 
                     inflow_df_fixed.set_index('date')], axis=1).reset_index()
df_Iseo = df_Iseo.add_suffix("_Iseo")
df_Iseo.rename(columns={'date_Iseo': 'date'}, inplace= True)

In [14]:
df_Iseo

,date,level_Iseo,release_Iseo,inflow_Iseo
0,2000-01-04,43.5,44.50,37.50
1,2000-01-05,42.5,43.28,35.58
2,2000-01-06,41.0,23.88,23.88
3,2000-01-07,41.0,43.38,34.28
4,2000-01-08,39.5,29.88,31.28
...,...,...,...,...
7297,2019-12-27,101.2,61.90,62.60
7298,2019-12-28,101.2,61.90,56.00
7299,2019-12-29,100.3,61.70,57.00
7300,2019-12-30,99.6,61.60,59.10


### Extract data from txt file (Como)

In [15]:
df_Como = pd.read_csv(lakes_folder + "lake_Como.txt", quotechar='"', delimiter = ' ', header = 0)
df_Como = df_Como.iloc[:-1]
df_Como.reset_index(inplace=True, drop=True)
df_Como["month"] = df_Como["month"].astype(int)
df_Como["year"] = df_Como["year"].astype(int)

In [16]:
df_Como.insert(0, "date", df_Como.apply(lambda x: pd.to_datetime(str(x['year']) + '-' + str(x['month']) + 
                                               '-' + str(x['day']), format='%Y-%m-%d'), axis = 1))
df_Como = df_Como.drop(["day", "month", "year"], axis=1)
df_Como = filter_date(df_Como)
df_Como = df_Como.add_suffix('_Como')
df_Como.rename(columns={'date_Como': 'date'}, inplace= True)

In [17]:
df_Como

,date,level_Como,release_Como,inflow_Como
0,2000-01-04,-0.020,90.00,81.6
1,2000-01-05,-0.025,86.40,94.8
2,2000-01-06,-0.020,85.90,69.1
3,2000-01-07,-0.030,85.90,94.3
4,2000-01-08,-0.025,85.50,51.9
...,...,...,...,...
7297,2019-12-27,0.795,192.20,167.0
7298,2019-12-28,0.780,190.32,156.8
7299,2019-12-29,0.760,189.24,122.1
7300,2019-12-30,0.720,189.43,155.9


### Put all together, compute weekly aggregations, save as csv

In [18]:
# the measures seem to be all in meters, maybe different places of measure

df_lakes = pd.concat([df_Como.set_index('date'), df_Maggiore.set_index('date'), 
                     df_Lugano.set_index('date'), df_Iseo.set_index('date')], axis=1).reset_index()
df_lakes

,date,level_Como,release_Como,inflow_Como,level_Maggiore,release_Maggiore,inflow_Maggiore,level_Lugano,release_Lugano,inflow_Lugano,level_Iseo,release_Iseo,inflow_Iseo
0,2000-01-04,-0.020,90.00,81.6,0.87,180.0,105.346867,270.4900,9.541417,7.277528,43.5,44.50,37.50
1,2000-01-05,-0.025,86.40,94.8,0.84,180.0,155.127079,270.4860,9.479354,7.781438,42.5,43.28,35.58
2,2000-01-06,-0.020,85.90,69.1,0.83,180.0,80.565532,270.4830,9.391882,8.825910,41.0,23.88,23.88
3,2000-01-07,-0.030,85.90,94.3,0.79,180.0,130.317045,270.4820,9.244486,5.848653,41.0,43.38,34.28
4,2000-01-08,-0.025,85.50,51.9,0.77,180.0,105.518352,270.4760,9.126597,8.560625,39.5,29.88,31.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7297,2019-12-27,0.795,192.20,167.0,1.26,376.9,162.597082,270.8885,55.850792,41.984472,101.2,61.90,62.60
7298,2019-12-28,0.780,190.32,156.8,1.19,360.5,583.274720,270.8640,51.665771,35.252576,101.2,61.90,56.00
7299,2019-12-29,0.760,189.24,122.1,1.28,360.5,335.363872,270.8350,48.688125,31.142986,100.3,61.70,57.00
7300,2019-12-30,0.720,189.43,155.9,1.27,355.0,328.781551,270.8040,39.226271,31.302660,99.6,61.60,59.10


In [19]:
# group by 8 days

tmp = df_lakes.copy()
tmp['group'] = (tmp.index // 8) + 1
tmp['date'] = tmp.groupby('group')['date'].transform('last')
df_lakes_8days = tmp.groupby('group').mean()
df_lakes_8days.insert(0, "date", tmp['date'].unique())
df_lakes_8days = df_lakes_8days.iloc[:-1,:]
df_lakes_8days.reset_index(inplace = True, drop = True)

df_lakes_8days

,date,level_Como,release_Como,inflow_Como,level_Maggiore,release_Maggiore,inflow_Maggiore,level_Lugano,release_Lugano,inflow_Lugano,level_Iseo,release_Iseo,inflow_Iseo
0,2000-01-11,-0.037500,85.92500,75.4250,0.77750,180.0000,117.910450,270.479500,9.261275,7.775598,40.4375,38.21000,32.17250
1,2000-01-19,-0.085000,84.03750,74.5875,0.59875,173.1250,123.414809,270.460500,8.307320,6.750897,32.8750,40.78500,33.52250
2,2000-01-27,-0.139375,83.50000,75.1000,0.45875,157.8125,135.863790,270.440875,7.458631,6.326687,24.0625,40.45625,31.79375
3,2000-02-04,-0.191250,83.63750,69.9875,0.38375,144.9375,122.806298,270.436000,7.018141,6.947394,14.5000,36.70375,30.49125
4,2000-02-12,-0.248125,72.55000,64.1500,0.33875,124.3750,111.810478,270.432625,6.710347,7.134826,7.8750,34.92875,28.62875
...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,2019-11-23,0.954375,328.33750,358.7500,1.22875,624.6375,792.028246,270.882000,78.530102,97.030319,70.3750,112.57500,111.20000
908,2019-12-01,0.955000,333.96500,293.0625,1.72750,922.5625,807.530962,270.879062,110.729393,84.942284,77.9000,90.16250,103.13750
909,2019-12-09,0.860000,182.55375,179.3750,1.35125,406.6750,405.563649,270.604125,46.087362,33.423734,86.0000,68.32500,69.35000
910,2019-12-17,0.780000,179.72875,166.0875,1.36750,344.6125,333.721877,270.495500,26.106333,29.289927,87.3625,52.60000,55.05000


In [20]:
# add weekly aggregations

df_lakes_8days['year'] = [date.year for date in df_lakes_8days['date']]
df_lakes_8days['week'] = [date.isocalendar()[1] for date in df_lakes_8days['date']]

last_training_date = '2009-12-29'
last_training_index = df_lakes_8days.index[df_lakes_8days['date'] == last_training_date][0]
train_df = df_lakes_8days.iloc[:last_training_index,:]

weekly_means_columns = train_df.groupby(['week']).mean().iloc[:, :-1]
index = df_lakes_8days.index
df_lakes_8days = pd.merge(df_lakes_8days, weekly_means_columns, how='left', on=['week'], 
                      suffixes=['','_weekoftheyear']).set_index(index)

lakes_features = df_lakes_8days.columns[1:13]
weekly_means_columns = df_lakes_8days.columns[15:]

for feature, weekly_mean in zip(lakes_features, weekly_means_columns):
  df_lakes_8days['cyclostat_' + feature] = df_lakes_8days[feature] - df_lakes_8days[weekly_mean]
  df_lakes_8days.drop(weekly_mean, axis='columns', inplace = True)
  df_lakes_8days.drop(feature, axis='columns', inplace = True)

max_aggreg = 24

for column in df_lakes_8days.columns[3:]:
    df_lakes_8days[column + '_1w'] = df_lakes_8days.apply(lambda x:df_lakes_8days.loc[(df_lakes_8days['date'] >= x.date+np.timedelta64(-8,'D')) & 
                                  (df_lakes_8days['date'] <= x.date), column].mean(), axis=1).values
    for i in range(4, max_aggreg+1, 4):
        if i != 20:
            df_lakes_8days[column + '_' + str(i) + 'w'] = df_lakes_8days.apply(lambda x:df_lakes_8days.loc[(df_lakes_8days['date']>= x.date+np.timedelta64(-i*8,'D')) & 
                                          (df_lakes_8days['date'] <= x.date), column].mean(), axis=1).values

In [21]:
df_lakes_8days

,date,year,week,cyclostat_level_Como,cyclostat_release_Como,cyclostat_inflow_Como,cyclostat_level_Maggiore,cyclostat_release_Maggiore,cyclostat_inflow_Maggiore,cyclostat_level_Lugano,...,cyclostat_release_Iseo_8w,cyclostat_release_Iseo_12w,cyclostat_release_Iseo_16w,cyclostat_release_Iseo_24w,cyclostat_inflow_Iseo_1w,cyclostat_inflow_Iseo_4w,cyclostat_inflow_Iseo_8w,cyclostat_inflow_Iseo_12w,cyclostat_inflow_Iseo_16w,cyclostat_inflow_Iseo_24w
0,2000-01-11,2000,2,-0.188062,-12.509125,-19.340000,-0.010937,15.325000,-26.778579,0.031600,...,-8.350000,-8.350000,-8.350000,-8.350000,-11.782875,-11.782875,-11.782875,-11.782875,-11.782875,-11.782875
1,2000-01-19,2000,3,-0.181607,-6.145536,-15.450000,0.049732,38.737500,-13.751624,0.010804,...,-4.991964,-4.991964,-4.991964,-4.991964,-9.042955,-9.042955,-9.042955,-9.042955,-9.042955,-9.042955
2,2000-01-27,2000,4,-0.145859,2.343750,-3.306250,-0.179844,40.951562,9.972556,-0.001781,...,-1.965997,-1.965997,-1.965997,-1.965997,-2.201362,-5.395199,-5.395199,-5.395199,-5.395199,-5.395199
3,2000-02-04,2000,5,-0.308875,-10.605250,-21.941250,-0.425375,8.527500,-19.424330,-0.039387,...,-2.883092,-2.883092,-2.883092,-2.883092,-3.273906,-6.158431,-6.158431,-6.158431,-6.158431,-6.158431
4,2000-02-12,2000,6,-0.345313,-14.937000,-21.767500,-0.467750,-10.973750,-18.373267,-0.030487,...,-3.227873,-3.227873,-3.227873,-3.227873,-7.276187,-6.147595,-6.147595,-6.147595,-6.147595,-6.147595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,2019-11-23,2019,47,0.611797,113.167812,172.976562,0.316828,222.983203,453.414253,0.335906,...,7.358874,6.618512,3.963780,3.651398,26.316391,22.291104,9.015633,8.703101,4.355792,4.328937
908,2019-12-01,2019,48,0.989375,243.211250,189.694643,1.171607,778.978571,624.114600,0.447866,...,12.662516,10.888303,7.031628,6.223670,49.369364,29.086389,15.361391,13.686606,9.071943,7.000155
909,2019-12-09,2019,50,0.523000,27.302000,38.967500,0.383700,95.205156,143.619652,0.055250,...,13.995777,11.304576,8.028341,7.645070,36.126161,31.671796,16.799117,15.245232,11.069811,8.173925
910,2019-12-17,2019,51,0.351696,45.127321,38.385714,0.176250,102.508929,79.725916,-0.018375,...,13.571928,10.404897,8.576684,7.586921,7.943750,25.418771,17.802232,14.004607,11.285564,8.166205


In [ ]:
df_lakes_8days.to_csv('./lakes/lakes_with_aggreg.csv')

### Split the lakes in the corresponding basins

In [ ]:
Ticino = df_lakes_8days.filter(like='Lugano')
Maggiore = df_lakes_8days.filter(like='Maggiore')
df_Ticino = pd.concat([df_lakes_8days.iloc[:, :3], Ticino, Maggiore], axis=1)
Adda = df_lakes_8days.filter(like='Como')
df_Adda = pd.concat([df_lakes_8days.iloc[:, :3], Adda], axis=1)
Oglio_Iseo = df_lakes_8days.filter(like='Iseo')
df_Oglio_Iseo = pd.concat([df_lakes_8days.iloc[:, :3], Oglio_Iseo], axis=1)

In [ ]:
df_Ticino.to_csv('./lakes/lakes_Ticino_with_aggreg.csv')
df_Adda.to_csv('./lakes/lakes_Adda_with_aggreg.csv')
df_Oglio_Iseo.to_csv('./lakes/lakes_Oglio_Iseo_with_aggreg.csv')